In [29]:
### importing libraries

from datetime import datetime,timedelta
from collections import defaultdict
from tqdm import tqdm

import numpy as np
import pandas as pd
import pickle as pkl

import json
import os
import gc

In [2]:
os.getcwd()

'/home/sidd/workspace/Student-Stress-Prediction'

In [3]:
with open('./data/ema_df_di.pkl' ,'rb') as f:
    ema_df_di = pkl.load(f)

len(ema_df_di)


11

In [4]:
ema_df_di.keys()

dict_keys(['Social', 'Class', 'Sleep', 'Activity', 'Exercise', 'Study Spaces', 'Events', 'Dining Halls', 'Lab', 'Mood', 'Class 2'])

In [5]:
ema_df_di['Social'].head()

,user_id,number_Social,date_time,year,month,day,hours,minute,weekday,weeknumber
0,u22,2,"04/02/2013, 11:22",2013,04,02,11,22,Tue,13
1,u22,3,"03/29/2013, 17:21",2013,03,29,17,21,Fri,12
2,u22,3,"03/29/2013, 21:41",2013,03,29,21,41,Fri,12
3,u22,1,"04/03/2013, 21:21",2013,04,03,21,21,Wed,13
4,u22,3,"04/03/2013, 18:14",2013,04,03,18,14,Wed,13


In [6]:
label_df = pd.read_csv('./data/label_after_EDA_df.csv')
label_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2542 entries, 0 to 2541
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   user_id       2542 non-null   object
 1   date_time     2542 non-null   object
 2   year          2542 non-null   int64 
 3   month         2542 non-null   int64 
 4   day           2542 non-null   int64 
 5   hours         2542 non-null   int64 
 6   minute        2542 non-null   int64 
 7   weekday       2542 non-null   object
 8   weeknumber    2542 non-null   int64 
 9   label_source  2542 non-null   object
 10  rating        2542 non-null   int64 
 11  stress_label  2542 non-null   int64 
dtypes: int64(8), object(4)
memory usage: 238.4+ KB


In [7]:
label_df.head()

,user_id,date_time,year,month,day,hours,minute,weekday,weeknumber,label_source,rating,stress_label
0,u00,2013-03-25 11:54:00,2013,3,25,11,54,Mon,12,stress_ema,2,1
1,u00,2013-03-25 20:33:00,2013,3,25,20,33,Mon,12,stress_ema,2,1
2,u00,2013-03-26 18:12:00,2013,3,26,18,12,Tue,12,stress_ema,1,0
3,u00,2013-03-27 19:25:00,2013,3,27,19,25,Wed,12,stress_ema,1,0
4,u00,2013-03-28 11:43:00,2013,3,28,11,43,Thu,12,stress_ema,2,1


In [8]:
label_df.tail()

,user_id,date_time,year,month,day,hours,minute,weekday,weeknumber,label_source,rating,stress_label
2537,u59,2013-06-05 17:06:00,2013,6,5,17,6,Wed,22,stress_ema,4,0
2538,u59,2013-06-06 03:21:00,2013,6,6,3,21,Thu,22,mood2_ema,3,0
2539,u59,2013-06-06 03:22:00,2013,6,6,3,22,Thu,22,stress_ema,4,0
2540,u59,2013-06-07 16:42:00,2013,6,7,16,42,Fri,22,mood2_ema,1,0
2541,u59,2013-06-07 16:46:00,2013,6,7,16,46,Fri,22,stress_ema,4,0


In [9]:
label_df['date_time'] = pd.to_datetime(label_df['date_time'])

In [10]:
label_df['date_time'].nunique(),len(label_df)

(2231, 2542)

In [11]:
# label_df.drop_duplicates(subset='date_time',inplace= True)
# label_df['stress_label'].value_counts()


# ################ DROPPING USER LEVEL DUPLICATES  BASED ON TIMESTAMPS ##############

drop_index_li = []

for u in label_df['user_id'].unique():
    tmp_df = label_df[label_df['user_id'] == u]
    # tmp_df.drop_duplicates(subset='date_time',inplace=True)
    tmp_li = list(label_df[label_df['user_id'] == u].date_time)

    for i in range(len(tmp_li)-1):

        if tmp_li[i] == tmp_li[i+1]:
            drop_index_li.extend(list(tmp_df[tmp_df['date_time'] == tmp_li[i+1]].index)[:-1])


len(set(drop_index_li)),len(label_df)



(87, 2542)

In [12]:
# len(set(tmp_li)),len(tmp_li)

In [13]:
# len(set(tmp_li)),len(tmp_li)
# c = 0
# s = 0
# for i in range(len(tmp_li)-1):
#     if tmp_li[i] == tmp_li[i+1]:
#         c+=1
#     if tmp_li[i] > tmp_li[i+1]:
#         s+=1
# print(c,s)

In [14]:
len(label_df),len(label_df.drop(drop_index_li,axis=0))

(2542, 2455)

In [15]:
label_pre_drop_df = label_df
label_df = label_df.drop(drop_index_li,axis=0)


In [16]:
len(label_df)

2455

In [17]:
# drop_index_li

dup_sum =0

for u in label_df['user_id'].unique():
    t_df = label_df[label_df['user_id'] == u ]
    uni_en = len(set(t_df['date_time']))
    en = len(t_df)
    if uni_en != en:
        print(u,uni_en,en)
        dup_sum += en-uni_en

dup_sum      

0

In [18]:
# label_df[label_df['user_id'] == 'u30']

In [19]:
label_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2455 entries, 0 to 2541
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   user_id       2455 non-null   object        
 1   date_time     2455 non-null   datetime64[ns]
 2   year          2455 non-null   int64         
 3   month         2455 non-null   int64         
 4   day           2455 non-null   int64         
 5   hours         2455 non-null   int64         
 6   minute        2455 non-null   int64         
 7   weekday       2455 non-null   object        
 8   weeknumber    2455 non-null   int64         
 9   label_source  2455 non-null   object        
 10  rating        2455 non-null   int64         
 11  stress_label  2455 non-null   int64         
dtypes: datetime64[ns](1), int64(8), object(3)
memory usage: 249.3+ KB


In [20]:
## writing label_df after dropping same entries wrt user and timestamp

label_df.to_csv('./data/label_df.csv',index= False)

In [21]:
# df_di = defaultdict(list)
# df_di['fi'].extend(['1','2','3','1','1'])
# df_di['se'].extend(['a','a','b','a','b'])


# ch_df = pd.DataFrame(df_di)
# ch_df

In [22]:
# ch_df.groupby('se').apply(lambda x : x.drop_duplicates(subset='fi'))

## Preparing EMA data


In [23]:
for k,v in ema_df_di.items():
    print(k,len(v))

Social 1169
Class 1097
Sleep 1390
Activity 538
Exercise 763
Study Spaces 200
Events 232
Dining Halls 214
Lab 142
Mood 277
Class 2 97


In [24]:
ema_df_di['Social']

,user_id,number_Social,date_time,year,month,day,hours,minute,weekday,weeknumber
0,u22,2,"04/02/2013, 11:22",2013,04,02,11,22,Tue,13
1,u22,3,"03/29/2013, 17:21",2013,03,29,17,21,Fri,12
2,u22,3,"03/29/2013, 21:41",2013,03,29,21,41,Fri,12
3,u22,1,"04/03/2013, 21:21",2013,04,03,21,21,Wed,13
4,u22,3,"04/03/2013, 18:14",2013,04,03,18,14,Wed,13
...,...,...,...,...,...,...,...,...,...,...
1164,u59,4,"05/26/2013, 12:17",2013,05,26,12,17,Sun,20
1165,u59,3,"06/05/2013, 17:05",2013,06,05,17,05,Wed,22
1166,u59,4,"06/01/2013, 23:00",2013,06,01,23,00,Sat,21
1167,u59,3,"06/04/2013, 16:18",2013,06,04,16,18,Tue,22


## Social

In [31]:
AGGREGATION_DAYS = 1

social_df = ema_df_di['Social']
social_df['date_time'] = pd.to_datetime(social_df['date_time'])


social_di = defaultdict(list)


for usr_id in tqdm(social_df['user_id'].unique()):
    
    # print(usr_id)
    
    tmp_df = social_df[social_df['user_id'] == usr_id]


    # time stamps
    
    tmp_li = list(label_df[label_df['user_id'] == usr_id].date_time)

    for time_b in tmp_li:
        
        time_a = time_b - timedelta(days=AGGREGATION_DAYS)
        slice_df = tmp_df[(time_a < tmp_df.date_time) & (tmp_df.date_time< time_b)]

        social_di['user_id'].append(usr_id)
        social_di['date_time'].append(time_b)
        social_di['number_Social'].append(slice_df['number_Social'].sum())

    gc.collect()



100%|██████████| 48/48 [00:05<00:00,  9.54it/s]


In [97]:
social_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1169 entries, 0 to 1168
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   user_id        1169 non-null   object        
 1   number_Social  1169 non-null   object        
 2   date_time      1169 non-null   datetime64[ns]
 3   year           1169 non-null   object        
 4   month          1169 non-null   object        
 5   day            1169 non-null   object        
 6   hours          1169 non-null   object        
 7   minute         1169 non-null   object        
 8   weekday        1169 non-null   object        
 9   weeknumber     1169 non-null   object        
dtypes: datetime64[ns](1), object(9)
memory usage: 91.5+ KB


In [32]:
for k,v in social_di.items():
    print(k,len(v))

user_id 2455
date_time 2455
number_Social 2455


In [33]:
social_mer_df = pd.DataFrame(social_di)

social_mer_df.head()

,user_id,date_time,number_Social
0,u22,2013-03-27 00:35:00,0
1,u22,2013-03-27 15:03:00,2
2,u22,2013-03-27 20:06:00,2
3,u22,2013-03-28 00:20:00,22
4,u22,2013-03-28 03:00:00,32


## Class

In [34]:
class_df = ema_df_di['Class']
class_df['date_time'] = pd.to_datetime(class_df['date_time'])
class_df.head()

,user_id,course_id_Class,experience_Class,due_Class,date_time,year,month,day,hours,minute,weekday,weeknumber
0,u25,cs65,3,1,2013-04-01 19:16:00,2013,04,01,19,16,Mon,13
1,u25,English 28,3,1,2013-03-27 17:10:00,2013,03,27,17,10,Wed,12
2,u25,computer science 65,3,2,2013-03-28 18:48:00,2013,03,28,18,48,Thu,12
3,u25,cs65,3,2,2013-03-29 17:14:00,2013,03,29,17,14,Fri,12
4,u25,no class today,1,2,2013-03-30 17:58:00,2013,03,30,17,58,Sat,12


In [79]:
AGGREGATION_DAYS = 1

class_di = defaultdict(list)


for usr_id in tqdm(label_df['user_id'].unique()):
    
    # print(usr_id)    
    tmp_df = class_df[class_df['user_id'] == usr_id]

    # time stamps
    tmp_li = list(label_df[label_df['user_id'] == usr_id].date_time)

    for time_b in tmp_li:
        
        time_a = time_b - timedelta(days=AGGREGATION_DAYS)
        slice_df = tmp_df[(time_a < tmp_df.date_time) & (tmp_df.date_time< time_b)]

        class_di['user_id'].append(usr_id)
        class_di['date_time'].append(time_b)

        tmp_di_1 = dict(slice_df['experience_Class'].value_counts())
        tmp_di_2 = dict(slice_df['due_Class'].value_counts())

        # if len(tmp_di_1) > 1:
        #     break

        class_di['experience_Class_neutral'].append(tmp_di_1.get('1',0))
        class_di['experience_Class_st_agree'].append(tmp_di_1.get('2',0))
        class_di['experience_Class_agree'].append(tmp_di_1.get('3',0))
        class_di['experience_Class_disagree'].append(tmp_di_1.get('4',0))
        class_di['experience_Class_st_disagree'].append(tmp_di_1.get('5',0))

        class_di['due_Class_yes'].append(tmp_di_2.get('1',0))
        class_di['due_Class_no'].append(tmp_di_2.get('2',0))

    gc.collect()


100%|██████████| 48/48 [00:06<00:00,  7.92it/s]


In [80]:
len(tmp_di_1)

0

In [81]:
slice_df

,user_id,course_id_Class,experience_Class,due_Class,date_time,year,month,day,hours,minute,weekday,weeknumber


In [82]:
for k,v in class_di.items():
    print(k,len(v))

user_id 2455
date_time 2455
experience_Class_neutral 2455
experience_Class_st_agree 2455
experience_Class_agree 2455
experience_Class_disagree 2455
experience_Class_st_disagree 2455
due_Class_yes 2455
due_Class_no 2455


In [83]:
class_mer_df = pd.DataFrame(class_di)
class_mer_df.head()

,user_id,date_time,experience_Class_neutral,experience_Class_st_agree,experience_Class_agree,experience_Class_disagree,experience_Class_st_disagree,due_Class_yes,due_Class_no
0,u00,2013-03-25 11:54:00,0,0,0,0,0,0,0
1,u00,2013-03-25 20:33:00,0,0,0,0,0,0,0
2,u00,2013-03-26 18:12:00,0,0,1,0,0,0,1
3,u00,2013-03-27 19:25:00,1,0,1,0,0,1,1
4,u00,2013-03-28 11:43:00,0,0,1,0,0,0,1


In [84]:
for col in class_mer_df.columns:
    if col != 'user_id' and col !='date_time':
        print(col, class_mer_df[col].nunique(),class_mer_df[col].unique())

experience_Class_neutral 4 [0 1 2 3]
experience_Class_st_agree 5 [0 1 2 3 4]
experience_Class_agree 7 [0 1 2 3 4 5 6]
experience_Class_disagree 3 [0 1 2]
experience_Class_st_disagree 3 [0 2 1]
due_Class_yes 6 [0 1 2 3 4 5]
due_Class_no 5 [0 1 2 3 4]


## Sleep

In [85]:
sleep_df = ema_df_di['Sleep']
sleep_df['date_time'] = pd.to_datetime(sleep_df['date_time'])
sleep_df.head()

,user_id,hour_Sleep,rate_Sleep,social_Sleep,date_time,year,month,day,hours,minute,weekday,weeknumber
0,u04,6,2,1,2013-04-01 08:03:00,2013,04,01,08,03,Mon,13
1,u04,6,2,1,2013-03-26 18:59:00,2013,03,26,18,59,Tue,12
2,u04,6,2,1,2013-03-27 11:18:00,2013,03,27,11,18,Wed,12
3,u04,6,2,1,2013-03-27 08:15:00,2013,03,27,08,15,Wed,12
4,u04,7,2,1,2013-03-28 08:13:00,2013,03,28,08,13,Thu,12


In [88]:
AGGREGATION_DAYS = 1

sleep_di = defaultdict(list)


for usr_id in tqdm(label_df['user_id'].unique()):
    
    # print(usr_id)    
    tmp_df = sleep_df[sleep_df['user_id'] == usr_id]

    # time stamps
    tmp_li = list(label_df[label_df['user_id'] == usr_id].date_time)

    for time_b in tmp_li:
        
        time_a = time_b - timedelta(days=AGGREGATION_DAYS)
        slice_df = tmp_df[(time_a < tmp_df.date_time) & (tmp_df.date_time< time_b)]

        sleep_di['user_id'].append(usr_id)
        sleep_di['date_time'].append(time_b)

        tmp_di_1 = dict(slice_df['hour_Sleep'].value_counts())
        tmp_di_2 = dict(slice_df['rate_Sleep'].value_counts())
        tmp_di_3 = dict(slice_df['social_Sleep'].value_counts())

        # if len(tmp_di_1)> 1:
        #     break
        sleep_di['hour_Sleep_less_than_3'].append(tmp_di_1.get(1,0))

        val_3_to_7 = sum([tmp_di_1.get(str(i),0) for i in range(2,9)])
        sleep_di['hour_Sleep_3_to_7'].append(val_3_to_7)

        val_7_to_9 = sum([tmp_di_1.get(str(i),0) for i in range(9,14)])
        sleep_di['hour_Sleep_7_to_9'].append(val_7_to_9)

        val_more_than_9 = sum([tmp_di_1.get(str(i),0) for i in range(14,20)])
        sleep_di['hour_Sleep_more_than_9'].append(val_more_than_9)

        
        sleep_di['rate_Sleep_v_good'].append(tmp_di_2.get('1',0))
        sleep_di['rate_Sleep_f_good'].append(tmp_di_2.get('2',0))
        sleep_di['rate_Sleep_f_bad'].append(tmp_di_2.get('3',0))
        sleep_di['rate_Sleep_v_bad'].append(tmp_di_2.get('4',0))
        
        sleep_di['social_Sleep_none'].append(tmp_di_3.get('1',0))
        sleep_di['social_Sleep_once'].append(tmp_di_3.get('2',0))
        sleep_di['social_Sleep_twice'].append(tmp_di_3.get('3',0))
        sleep_di['social_Sleep_trice_more'].append(tmp_di_3.get('4',0))


    gc.collect()


100%|██████████| 48/48 [00:06<00:00,  7.29it/s]


In [89]:
tmp_di_1

{}

In [90]:
for k,v in sleep_di.items():
    print(k,len(v))

user_id 2455
date_time 2455
hour_Sleep_less_than_3 2455
hour_Sleep_3_to_7 2455
hour_Sleep_7_to_9 2455
hour_Sleep_more_than_9 2455
rate_Sleep_v_good 2455
rate_Sleep_f_good 2455
rate_Sleep_f_bad 2455
rate_Sleep_v_bad 2455
social_Sleep_none 2455
social_Sleep_once 2455
social_Sleep_twice 2455
social_Sleep_trice_more 2455


In [91]:
sleep_mer_df = pd.DataFrame(sleep_di)
sleep_mer_df.tail()

,user_id,date_time,hour_Sleep_less_than_3,hour_Sleep_3_to_7,hour_Sleep_7_to_9,hour_Sleep_more_than_9,rate_Sleep_v_good,rate_Sleep_f_good,rate_Sleep_f_bad,rate_Sleep_v_bad,social_Sleep_none,social_Sleep_once,social_Sleep_twice,social_Sleep_trice_more
2450,u59,2013-06-05 17:06:00,0,0,0,0,0,0,0,0,0,0,0,0
2451,u59,2013-06-06 03:21:00,0,0,0,1,1,0,0,0,1,0,0,0
2452,u59,2013-06-06 03:22:00,0,0,0,1,1,0,0,0,1,0,0,0
2453,u59,2013-06-07 16:42:00,0,0,0,0,0,0,0,0,0,0,0,0
2454,u59,2013-06-07 16:46:00,0,0,0,0,0,0,0,0,0,0,0,0


In [92]:
sleep_mer_df['hour_Sleep_more_than_9'].nunique()

3

In [107]:
for col in sleep_mer_df.columns:
    if col != 'user_id' and col !='date_time':
        print(col, sleep_mer_df[col].nunique(),sleep_mer_df[col].unique())

hour_Sleep_less_than_3 1 [0]
hour_Sleep_3_to_7 5 [0 1 2 3 4]
hour_Sleep_7_to_9 3 [0 1 2]
hour_Sleep_more_than_9 3 [0 1 2]
rate_Sleep_v_good 4 [0 1 2 3]
rate_Sleep_f_good 5 [0 1 2 3 4]
rate_Sleep_f_bad 3 [0 1 2]
rate_Sleep_v_bad 3 [0 1 2]
social_Sleep_none 5 [0 1 2 3 4]
social_Sleep_once 5 [0 1 2 3 4]
social_Sleep_twice 3 [0 1 2]
social_Sleep_trice_more 3 [0 1 2]


## activity

In [93]:
activity_df = ema_df_di['Activity']
activity_df['date_time'] = pd.to_datetime(activity_df['date_time'])
activity_df.head()

,user_id,working_Activity,relaxing_Activity,other_working_Activity,other_relaxing_Activity,date_time,year,month,day,hours,minute,weekday,weeknumber
0,u02,2,1,2,2,2013-04-07 22:16:00,2013,04,07,22,16,Sun,13
1,u02,3,1,3,1,2013-04-10 02:45:00,2013,04,10,02,45,Wed,14
2,u02,2,1,3,1,2013-04-10 23:00:00,2013,04,10,23,00,Wed,14
3,u02,2,1,3,2,2013-04-11 23:02:00,2013,04,11,23,02,Thu,14
4,u02,3,1,3,1,2013-04-14 23:01:00,2013,04,14,23,01,Sun,14


In [98]:
AGGREGATION_DAYS = 1

activity_di = defaultdict(list)


for usr_id in tqdm(label_df['user_id'].unique()):
    
    # print(usr_id)    
    tmp_df = activity_df[activity_df['user_id'] == usr_id]

    # time stamps
    tmp_li = list(label_df[label_df['user_id'] == usr_id].date_time)

    for time_b in tmp_li:
        
        time_a = time_b - timedelta(days=AGGREGATION_DAYS)
        slice_df = tmp_df[(time_a < tmp_df.date_time) & (tmp_df.date_time< time_b)]

        activity_di['user_id'].append(usr_id)
        activity_di['date_time'].append(time_b)

        tmp_di_1 = dict(slice_df['working_Activity'].value_counts())
        tmp_di_2 = dict(slice_df['relaxing_Activity'].value_counts())
        tmp_di_3 = dict(slice_df['other_working_Activity'].value_counts())
        tmp_di_4 = dict(slice_df['other_relaxing_Activity'].value_counts())
        
        val_less_than_50 = sum([ tmp_di_1.get(str(i),0) for i in range(1,4)])
        val_more_than_50 = sum([ tmp_di_1.get(str(i),0) for i in range(4,6)])

        activity_di['working_Activity_alone_less_50'].append(val_less_than_50)
        activity_di['working_Activity_alone_more_50'].append(val_more_than_50)

        val_less_than_50 = sum([ tmp_di_2.get(str(i),0) for i in range(1,4)])
        val_more_than_50 = sum([ tmp_di_2.get(str(i),0) for i in range(4,6)])

        activity_di['relaxing_Activity_alone_less_50'].append(val_less_than_50)
        activity_di['relaxing_Activity_alone_more_50'].append(val_more_than_50)

        val_less_than_50 = sum([ tmp_di_3.get(str(i),0) for i in range(1,4)])
        val_more_than_50 = sum([ tmp_di_3.get(str(i),0) for i in range(4,6)])

        activity_di['other_working_Activity_less_50'].append(val_less_than_50)
        activity_di['other_working_Activity_more_50'].append(val_more_than_50)

        val_less_than_50 = sum([ tmp_di_4.get(str(i),0) for i in range(1,4)])
        val_more_than_50 = sum([ tmp_di_4.get(str(i),0) for i in range(4,6)])

        activity_di['other_relaxing_Activity_less_50'].append(val_less_than_50)
        activity_di['other_relaxing_Activity_more_50'].append(val_more_than_50)



    gc.collect()


100%|██████████| 48/48 [00:07<00:00,  6.60it/s]


In [103]:
for k,v in activity_di.items():
    print(k,len(v))

user_id 2455
date_time 2455
working_Activity_alone_less_50 2455
working_Activity_alone_more_50 2455
relaxing_Activity_alone_less_50 2455
relaxing_Activity_alone_more_50 2455
other_working_Activity_less_50 2455
other_working_Activity_more_50 2455
other_relaxing_Activity_less_50 2455
other_relaxing_Activity_more_50 2455


In [104]:
activity_mer_df = pd.DataFrame(activity_di)
activity_mer_df.head()

,user_id,date_time,working_Activity_alone_less_50,working_Activity_alone_more_50,relaxing_Activity_alone_less_50,relaxing_Activity_alone_more_50,other_working_Activity_less_50,other_working_Activity_more_50,other_relaxing_Activity_less_50,other_relaxing_Activity_more_50
0,u00,2013-03-25 11:54:00,0,0,0,0,0,0,0,0
1,u00,2013-03-25 20:33:00,0,0,0,0,0,0,0,0
2,u00,2013-03-26 18:12:00,0,0,0,0,0,0,0,0
3,u00,2013-03-27 19:25:00,0,0,0,0,0,0,0,0
4,u00,2013-03-28 11:43:00,0,0,0,0,0,0,0,0


In [106]:
for col in activity_mer_df.columns:
    if col != 'user_id' and col !='date_time':
        print(col, activity_mer_df[col].nunique(),activity_mer_df[col].unique())

working_Activity_alone_less_50 4 [0 2 1 3]
working_Activity_alone_more_50 3 [0 1 2]
relaxing_Activity_alone_less_50 4 [0 2 1 3]
relaxing_Activity_alone_more_50 2 [0 1]
other_working_Activity_less_50 4 [0 1 2 3]
other_working_Activity_more_50 3 [0 1 2]
other_relaxing_Activity_less_50 3 [0 2 1]
other_relaxing_Activity_more_50 3 [0 1 2]


## Exercise

In [108]:
exercise_df = ema_df_di['Exercise']
exercise_df['date_time'] = pd.to_datetime(exercise_df['date_time'])
exercise_df.head()

,user_id,have_Exercise,schedule_Exercise,exercise_Exercise,walk_Exercise,date_time,year,month,day,hours,minute,weekday,weeknumber
0,u03,2,2,2,2,2013-04-05 10:29:00,2013,04,05,10,29,Fri,13
1,u03,2,2,2,2,2013-04-07 11:12:00,2013,04,07,11,12,Sun,13
2,u03,1,2,2,2,2013-04-11 12:44:00,2013,04,11,12,44,Thu,14
3,u03,2,2,1,2,2013-04-21 21:00:00,2013,04,21,21,00,Sun,15
4,u03,2,2,2,2,2013-04-28 20:20:00,2013,04,28,20,20,Sun,16


In [109]:
AGGREGATION_DAYS = 1

exercise_di = defaultdict(list)


for usr_id in tqdm(label_df['user_id'].unique()):
    
    # print(usr_id)    
    tmp_df = exercise_df[exercise_df['user_id'] == usr_id]

    # time stamps
    tmp_li = list(label_df[label_df['user_id'] == usr_id].date_time)

    for time_b in tmp_li:
        
        time_a = time_b - timedelta(days=AGGREGATION_DAYS)
        slice_df = tmp_df[(time_a < tmp_df.date_time) & (tmp_df.date_time< time_b)]

        exercise_di['user_id'].append(usr_id)
        exercise_di['date_time'].append(time_b)

        tmp_di_1 = dict(slice_df['have_Exercise'].value_counts())
        tmp_di_2 = dict(slice_df['schedule_Exercise'].value_counts())
        tmp_di_3 = dict(slice_df['exercise_Exercise'].value_counts())
        tmp_di_4 = dict(slice_df['walk_Exercise'].value_counts())
        

        exercise_di['have_Exercise_yes'].append(tmp_di_1.get('1',0))
        exercise_di['have_Exercise_no'].append(tmp_di_1.get('2',0))


        exercise_di['schedule_Exercise_yes'].append(tmp_di_2.get('1',0))
        exercise_di['schedule_Exercise_no'].append(tmp_di_2.get('2',0))

        val_less_than_hour = sum([ tmp_di_3.get(str(i),0) for i in range(2,4)])
        val_more_than_hour = sum([ tmp_di_3.get(str(i),0) for i in range(4,6)])

        exercise_di['exercise_Exercise_none'].append(tmp_di_3.get('1',0))
        exercise_di['exercise_Exercise_less_hour'].append(val_less_than_hour)
        exercise_di['exercise_Exercise_more_hour'].append(val_more_than_hour)

        val_less_than_hour = sum([ tmp_di_4.get(str(i),0) for i in range(2,4)])
        val_more_than_hour = sum([ tmp_di_4.get(str(i),0) for i in range(4,6)])

        exercise_di['walk_Exercise_none'].append(tmp_di_4.get('1',0))
        exercise_di['walk_Exercise_less_hour'].append(val_less_than_hour)
        exercise_di['walk_Exercise_more_hour'].append(val_more_than_hour)


    gc.collect()


100%|██████████| 48/48 [00:07<00:00,  6.61it/s]


In [110]:
for k,v in exercise_di.items():
    print(k,len(v))

user_id 2455
date_time 2455
have_Exercise_yes 2455
have_Exercise_no 2455
schedule_Exercise_yes 2455
schedule_Exercise_no 2455
exercise_Exercise_none 2455
exercise_Exercise_less_hour 2455
exercise_Exercise_more_hour 2455
walk_Exercise_none 2455
walk_Exercise_less_hour 2455
walk_Exercise_more_hour 2455


In [111]:
exercise_mer_df = pd.DataFrame(exercise_di)
exercise_mer_df.head()

,user_id,date_time,have_Exercise_yes,have_Exercise_no,schedule_Exercise_yes,schedule_Exercise_no,exercise_Exercise_none,exercise_Exercise_less_hour,exercise_Exercise_more_hour,walk_Exercise_none,walk_Exercise_less_hour,walk_Exercise_more_hour
0,u00,2013-03-25 11:54:00,0,0,0,0,0,0,0,0,0,0
1,u00,2013-03-25 20:33:00,0,0,0,0,0,0,0,0,0,0
2,u00,2013-03-26 18:12:00,0,0,0,0,0,0,0,0,0,0
3,u00,2013-03-27 19:25:00,0,0,0,0,0,0,0,0,0,0
4,u00,2013-03-28 11:43:00,0,0,0,0,0,0,0,0,0,0


In [112]:
for col in exercise_mer_df.columns:
    if col != 'user_id' and col !='date_time':
        print(col, exercise_mer_df[col].nunique(),exercise_mer_df[col].unique())

have_Exercise_yes 3 [0 1 2]
have_Exercise_no 4 [0 2 1 3]
schedule_Exercise_yes 3 [0 2 1]
schedule_Exercise_no 4 [0 1 2 3]
exercise_Exercise_none 4 [0 1 2 3]
exercise_Exercise_less_hour 3 [0 1 2]
exercise_Exercise_more_hour 3 [0 1 2]
walk_Exercise_none 3 [0 1 2]
walk_Exercise_less_hour 4 [0 1 2 3]
walk_Exercise_more_hour 3 [0 1 2]


## study space

## events

## dinning halls

## Lab

## Mood

In [113]:
mood_df = ema_df_di['Mood']
mood_df['date_time'] = pd.to_datetime(mood_df['date_time'])
mood_df.head()

,user_id,happyornot_Mood,happy_Mood,sadornot_Mood,sad_Mood,date_time,year,month,day,hours,minute,weekday,weeknumber
0,u02,2,1,1,1,2013-05-19 19:03:00,2013,05,19,19,03,Sun,19
1,u12,1,1,2,1,2013-05-04 15:30:00,2013,05,04,15,30,Sat,17
2,u53,1,2,2,1,2013-04-25 01:48:00,2013,04,25,01,48,Thu,16
3,u53,1,2,2,1,2013-05-17 03:49:00,2013,05,17,03,49,Fri,19
4,u53,2,1,1,1,2013-05-14 23:31:00,2013,05,14,23,31,Tue,19


In [114]:
AGGREGATION_DAYS = 1

mood_di = defaultdict(list)


for usr_id in tqdm(label_df['user_id'].unique()):
    
    # print(usr_id)    
    tmp_df = mood_df[mood_df['user_id'] == usr_id]

    # time stamps
    tmp_li = list(label_df[label_df['user_id'] == usr_id].date_time)

    for time_b in tmp_li:
        
        time_a = time_b - timedelta(days=AGGREGATION_DAYS)
        slice_df = tmp_df[(time_a < tmp_df.date_time) & (tmp_df.date_time< time_b)]

        mood_di['user_id'].append(usr_id)
        mood_di['date_time'].append(time_b)

        tmp_di_1 = dict(slice_df['happyornot_Mood'].value_counts())
        # tmp_di_2 = dict(slice_df['happy_Mood'].value_counts())
        tmp_di_3 = dict(slice_df['sadornot_Mood'].value_counts())
        # tmp_di_4 = dict(slice_df['sad_Mood'].value_counts())
        

        mood_di['happyornot_Mood_yes'].append(tmp_di_1.get('1',0))
        mood_di['happyornot_Mood_no'].append(tmp_di_1.get('2',0))


        mood_di['sadornot_Mood_yes'].append(tmp_di_3.get('1',0))
        mood_di['sadornot_Mood_no'].append(tmp_di_3.get('2',0))


    gc.collect()


100%|██████████| 48/48 [00:06<00:00,  7.71it/s]


In [115]:
for k,v in mood_di.items():
    print(k,len(v))

user_id 2455
date_time 2455
happyornot_Mood_yes 2455
happyornot_Mood_no 2455
sadornot_Mood_yes 2455
sadornot_Mood_no 2455


In [116]:
mood_mer_df = pd.DataFrame(mood_di)
mood_mer_df.head()

,user_id,date_time,happyornot_Mood_yes,happyornot_Mood_no,sadornot_Mood_yes,sadornot_Mood_no
0,u00,2013-03-25 11:54:00,0,0,0,0
1,u00,2013-03-25 20:33:00,0,0,0,0
2,u00,2013-03-26 18:12:00,0,0,0,0
3,u00,2013-03-27 19:25:00,0,0,0,0
4,u00,2013-03-28 11:43:00,0,0,0,0


In [117]:
for col in mood_mer_df.columns:
    if col != 'user_id' and col !='date_time':
        print(col, mood_mer_df[col].nunique(),mood_mer_df[col].unique())

happyornot_Mood_yes 5 [0 1 2 3 4]
happyornot_Mood_no 3 [0 1 2]
sadornot_Mood_yes 5 [0 2 1 3 4]
sadornot_Mood_no 5 [0 1 3 4 2]


## Class 2

In [119]:
class2_df = ema_df_di['Class 2']
class2_df['date_time'] = pd.to_datetime(class2_df['date_time'])
class2_df.head()

,user_id,challenge_Class 2,effort_Class 2,grade_Class 2,date_time,year,month,day,hours,minute,weekday,weeknumber
0,u42,1,2,3,2013-05-01 17:34:00,2013,05,01,17,34,Wed,17
1,u42,1,2,3,2013-05-02 16:33:00,2013,05,02,16,33,Thu,17
2,u42,3,1,3,2013-05-06 16:13:00,2013,05,06,16,13,Mon,18
3,u36,1,1,1,2013-04-30 16:04:00,2013,04,30,16,04,Tue,17
4,u36,1,1,1,2013-05-01 16:43:00,2013,05,01,16,43,Wed,17


In [121]:
AGGREGATION_DAYS = 1

class2_di = defaultdict(list)


for usr_id in tqdm(label_df['user_id'].unique()):
    
    # print(usr_id)    
    tmp_df = class2_df[class2_df['user_id'] == usr_id]

    # time stamps
    tmp_li = list(label_df[label_df['user_id'] == usr_id].date_time)

    for time_b in tmp_li:
        
        time_a = time_b - timedelta(days=AGGREGATION_DAYS)
        slice_df = tmp_df[(time_a < tmp_df.date_time) & (tmp_df.date_time< time_b)]

        class2_di['user_id'].append(usr_id)
        class2_di['date_time'].append(time_b)

        tmp_di_1 = dict(slice_df['challenge_Class 2'].value_counts())
        tmp_di_2 = dict(slice_df['effort_Class 2'].value_counts())
        tmp_di_3 = dict(slice_df['grade_Class 2'].value_counts())
        
        val_yes = sum([tmp_di_1.get(str(i),0) for i in range(1,4)])
        val_no = sum([tmp_di_1.get(str(i),0) for i in range(4,7)])

        class2_di['challenge_Class 2 yes'].append(val_yes)
        class2_di['challenge_Class 2 no'].append(val_no)

        val_yes = sum([tmp_di_2.get(str(i),0) for i in range(1,4)])
        val_no = sum([tmp_di_2.get(str(i),0) for i in range(4,7)])

        class2_di['effort_Class 2 yes'].append(val_yes)
        class2_di['effort_Class 2 no'].append(val_no)


        class2_di['grade_A'].append(tmp_di_3.get('1',0))
        class2_di['grade_A_min'].append(tmp_di_3.get('2',0))
        class2_di['grade_B_plu'].append(tmp_di_3.get('3',0))
        class2_di['grade_B'].append(tmp_di_3.get('4',0))
        class2_di['grade_B_min'].append(tmp_di_3.get('5',0))
        class2_di['grade_C_plu'].append(tmp_di_3.get('6',0))
        class2_di['grade_C'].append(tmp_di_3.get('7',0))
        class2_di['grade_C_min'].append(tmp_di_3.get('8',0))
        
    gc.collect()


100%|██████████| 48/48 [00:06<00:00,  7.15it/s]


In [122]:
for k,v in class2_di.items():
    print(k,len(v))

user_id 2455
date_time 2455
challenge_Class 2 yes 2455
challenge_Class 2 no 2455
effort_Class 2 yes 2455
effort_Class 2 no 2455
grade_A 2455
grade_A_min 2455
grade_B_plu 2455
grade_B 2455
grade_B_min 2455
grade_C_plu 2455
grade_C 2455
grade_C_min 2455


In [123]:
class2_mer_df = pd.DataFrame(class2_di)
class2_mer_df.head()

,user_id,date_time,challenge_Class 2 yes,challenge_Class 2 no,effort_Class 2 yes,effort_Class 2 no,grade_A,grade_A_min,grade_B_plu,grade_B,grade_B_min,grade_C_plu,grade_C,grade_C_min
0,u00,2013-03-25 11:54:00,0,0,0,0,0,0,0,0,0,0,0,0
1,u00,2013-03-25 20:33:00,0,0,0,0,0,0,0,0,0,0,0,0
2,u00,2013-03-26 18:12:00,0,0,0,0,0,0,0,0,0,0,0,0
3,u00,2013-03-27 19:25:00,0,0,0,0,0,0,0,0,0,0,0,0
4,u00,2013-03-28 11:43:00,0,0,0,0,0,0,0,0,0,0,0,0


In [124]:
for col in class2_mer_df.columns:
    if col != 'user_id' and col !='date_time':
        print(col, class2_mer_df[col].nunique(),class2_mer_df[col].unique())

challenge_Class 2 yes 4 [0 1 2 3]
challenge_Class 2 no 2 [0 1]
effort_Class 2 yes 4 [0 1 2 3]
effort_Class 2 no 2 [0 1]
grade_A 4 [0 1 2 3]
grade_A_min 2 [0 1]
grade_B_plu 2 [0 1]
grade_B 2 [0 1]
grade_B_min 2 [0 1]
grade_C_plu 1 [0]
grade_C 1 [0]
grade_C_min 1 [0]


## Merging EMA data

In [125]:
merged_ema_df = social_mer_df.merge(class_mer_df,on=['user_id','date_time']) 

merged_ema_df.head()

,user_id,date_time,number_Social,experience_Class_neutral,experience_Class_st_agree,experience_Class_agree,experience_Class_disagree,experience_Class_st_disagree,due_Class_yes,due_Class_no
0,u22,2013-03-27 00:35:00,0,0,0,0,0,0,0,0
1,u22,2013-03-27 15:03:00,2,1,0,0,0,0,0,1
2,u22,2013-03-27 20:06:00,2,2,0,1,0,0,0,3
3,u22,2013-03-28 00:20:00,22,2,0,1,0,0,0,3
4,u22,2013-03-28 03:00:00,32,2,0,1,0,0,0,3


In [127]:
len(merged_ema_df),len(merged_ema_df.columns)

(2455, 10)

In [128]:
merged_ema_df = merged_ema_df.merge(sleep_mer_df,on=['user_id','date_time'])  

merged_ema_df.head()

,user_id,date_time,number_Social,experience_Class_neutral,experience_Class_st_agree,experience_Class_agree,experience_Class_disagree,experience_Class_st_disagree,due_Class_yes,due_Class_no,...,hour_Sleep_7_to_9,hour_Sleep_more_than_9,rate_Sleep_v_good,rate_Sleep_f_good,rate_Sleep_f_bad,rate_Sleep_v_bad,social_Sleep_none,social_Sleep_once,social_Sleep_twice,social_Sleep_trice_more
0,u22,2013-03-27 00:35:00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,u22,2013-03-27 15:03:00,2,1,0,0,0,0,0,1,...,0,0,0,0,1,0,1,0,0,0
2,u22,2013-03-27 20:06:00,2,2,0,1,0,0,0,3,...,0,0,0,0,1,0,1,0,0,0
3,u22,2013-03-28 00:20:00,22,2,0,1,0,0,0,3,...,0,0,0,0,2,0,2,0,0,0
4,u22,2013-03-28 03:00:00,32,2,0,1,0,0,0,3,...,0,0,0,0,1,0,1,0,0,0


In [129]:
len(merged_ema_df),len(merged_ema_df.columns)

(2455, 22)

In [130]:
merged_ema_df = merged_ema_df.merge(activity_mer_df,on=['user_id','date_time'])   

merged_ema_df.head()

,user_id,date_time,number_Social,experience_Class_neutral,experience_Class_st_agree,experience_Class_agree,experience_Class_disagree,experience_Class_st_disagree,due_Class_yes,due_Class_no,...,social_Sleep_twice,social_Sleep_trice_more,working_Activity_alone_less_50,working_Activity_alone_more_50,relaxing_Activity_alone_less_50,relaxing_Activity_alone_more_50,other_working_Activity_less_50,other_working_Activity_more_50,other_relaxing_Activity_less_50,other_relaxing_Activity_more_50
0,u22,2013-03-27 00:35:00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,u22,2013-03-27 15:03:00,2,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,u22,2013-03-27 20:06:00,2,2,0,1,0,0,0,3,...,0,0,0,0,0,0,0,0,0,0
3,u22,2013-03-28 00:20:00,22,2,0,1,0,0,0,3,...,0,0,0,0,0,0,0,0,0,0
4,u22,2013-03-28 03:00:00,32,2,0,1,0,0,0,3,...,0,0,0,0,0,0,0,0,0,0


In [131]:
len(merged_ema_df)

2455

In [132]:
merged_ema_df = merged_ema_df.merge(exercise_mer_df,on=['user_id','date_time'])    

merged_ema_df.head()

,user_id,date_time,number_Social,experience_Class_neutral,experience_Class_st_agree,experience_Class_agree,experience_Class_disagree,experience_Class_st_disagree,due_Class_yes,due_Class_no,...,have_Exercise_yes,have_Exercise_no,schedule_Exercise_yes,schedule_Exercise_no,exercise_Exercise_none,exercise_Exercise_less_hour,exercise_Exercise_more_hour,walk_Exercise_none,walk_Exercise_less_hour,walk_Exercise_more_hour
0,u22,2013-03-27 00:35:00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,u22,2013-03-27 15:03:00,2,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,u22,2013-03-27 20:06:00,2,2,0,1,0,0,0,3,...,0,0,0,0,0,0,0,0,0,0
3,u22,2013-03-28 00:20:00,22,2,0,1,0,0,0,3,...,0,0,0,0,0,0,0,0,0,0
4,u22,2013-03-28 03:00:00,32,2,0,1,0,0,0,3,...,0,0,0,0,0,0,0,0,0,0


In [133]:
len(merged_ema_df)

2455

In [134]:
merged_ema_df = merged_ema_df.merge(mood_mer_df,on=['user_id','date_time'])     

merged_ema_df.head()

,user_id,date_time,number_Social,experience_Class_neutral,experience_Class_st_agree,experience_Class_agree,experience_Class_disagree,experience_Class_st_disagree,due_Class_yes,due_Class_no,...,exercise_Exercise_none,exercise_Exercise_less_hour,exercise_Exercise_more_hour,walk_Exercise_none,walk_Exercise_less_hour,walk_Exercise_more_hour,happyornot_Mood_yes,happyornot_Mood_no,sadornot_Mood_yes,sadornot_Mood_no
0,u22,2013-03-27 00:35:00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,u22,2013-03-27 15:03:00,2,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,u22,2013-03-27 20:06:00,2,2,0,1,0,0,0,3,...,0,0,0,0,0,0,0,0,0,0
3,u22,2013-03-28 00:20:00,22,2,0,1,0,0,0,3,...,0,0,0,0,0,0,0,0,0,0
4,u22,2013-03-28 03:00:00,32,2,0,1,0,0,0,3,...,0,0,0,0,0,0,0,0,0,0


In [135]:
len(merged_ema_df)

2455

In [136]:
merged_ema_df = merged_ema_df.merge(class2_mer_df,on=['user_id','date_time'])     

merged_ema_df.head()

,user_id,date_time,number_Social,experience_Class_neutral,experience_Class_st_agree,experience_Class_agree,experience_Class_disagree,experience_Class_st_disagree,due_Class_yes,due_Class_no,...,effort_Class 2 yes,effort_Class 2 no,grade_A,grade_A_min,grade_B_plu,grade_B,grade_B_min,grade_C_plu,grade_C,grade_C_min
0,u22,2013-03-27 00:35:00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,u22,2013-03-27 15:03:00,2,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,u22,2013-03-27 20:06:00,2,2,0,1,0,0,0,3,...,0,0,0,0,0,0,0,0,0,0
3,u22,2013-03-28 00:20:00,22,2,0,1,0,0,0,3,...,0,0,0,0,0,0,0,0,0,0
4,u22,2013-03-28 03:00:00,32,2,0,1,0,0,0,3,...,0,0,0,0,0,0,0,0,0,0


In [137]:
len(merged_ema_df)

2455

In [138]:
merged_ema_df = merged_ema_df.merge(label_df,on=['user_id','date_time'])     

merged_ema_df.head()

,user_id,date_time,number_Social,experience_Class_neutral,experience_Class_st_agree,experience_Class_agree,experience_Class_disagree,experience_Class_st_disagree,due_Class_yes,due_Class_no,...,year,month,day,hours,minute,weekday,weeknumber,label_source,rating,stress_label
0,u22,2013-03-27 00:35:00,0,0,0,0,0,0,0,0,...,2013,3,27,0,35,Wed,12,stress_ema,1,0
1,u22,2013-03-27 15:03:00,2,1,0,0,0,0,0,1,...,2013,3,27,15,3,Wed,12,stress_ema,1,0
2,u22,2013-03-27 20:06:00,2,2,0,1,0,0,0,3,...,2013,3,27,20,6,Wed,12,stress_ema,4,0
3,u22,2013-03-28 00:20:00,22,2,0,1,0,0,0,3,...,2013,3,28,0,20,Thu,12,stress_ema,5,0
4,u22,2013-03-28 03:00:00,32,2,0,1,0,0,0,3,...,2013,3,28,3,0,Thu,12,stress_ema,5,0


In [139]:
len(merged_ema_df)

2455

## Writing EMA data to csv

In [140]:
merged_ema_df.to_csv('./data/ema_data.csv',index=False)